In [36]:
# import tensorflow as tf
import time
import numpy as np
import os
import copy
import pickle
import argparse
import utility
import pandas as pd
from sklearn.metrics import *



2023-03-13 10:50:16.948013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 10:50:17.800939: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [1]:
import pandas as pd


In [28]:
train_df = pd.read_pickle(r'ml1m-2/training_df.pkl')    
vali_df = pd.read_pickle(r'ml1m-2/valiing_df.pkl')   # for validation
# vali_df = pickle.load(open('./' + dataname + '/testing_df.pkl'))  # for testing
key_genre = pd.read_pickle(r'ml1m-2/key_genre.pkl')  
item_idd_genre_list = pd.read_pickle(r'ml1m-2/item_idd_genre_list.pkl')   
genre_item_vector = pd.read_pickle(r'ml1m-2/genre_item_vector.pkl')    
genre_count = pd.read_pickle(r'ml1m-2/genre_count.pkl')      
user_genre_count = pd.read_pickle(r'ml1m-2/user_genre_count.pkl') 

num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_genre = len(key_genre)

item_genre_list = []
for u in range(num_item):
    gl = item_idd_genre_list[u]
    tmp = []
    for g in gl:
        if g in key_genre:
            tmp.append(g)
    item_genre_list.append(tmp)

In [18]:
train_df = train_df[train_df['user_id'].isin(vali_df['user_id'].unique())]
train_df = train_df[train_df['item_id'].isin(vali_df['item_id'].unique())]

In [29]:
def create_user_list(df, user_size):
    user_list = [list() for u in range(user_size)]
    for row in df.itertuples():
        user_list[row.user_id].append(row.item_id)
    return user_list

def create_pair(user_list):
    pair = []
    for user, item_list in enumerate(user_list):
        pair.extend([(user, item) for item in item_list])
    return pair

In [30]:
train_df

,user_id,item_id
0,0,0
1,0,2
2,1,4
3,1,5
4,1,6
...,...,...
152519,5561,168
152520,5561,100
152521,5561,171
152522,5561,102


In [31]:
vali_df

,user_id,item_id
0,0,1
1,1,3
2,1,7
3,3,5
4,3,2
...,...,...
21005,5561,17
21006,5561,46
21007,5561,25
21008,5561,0


In [32]:
train_user_list = create_user_list(train_df, num_user)
test_user_list = create_user_list(vali_df, num_user)
train_pair = create_pair(train_user_list)


In [33]:
dataset = {'user_size': num_user, 'item_size': num_item,
            'train_user_list': train_user_list, 'test_user_list': test_user_list,
            'train_pair': train_pair}

In [34]:
dataset

{'user_size': 5562,
 'item_size': 543,
 'train_user_list': [[0, 2],
  [4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 19, 20],
  [5, 21, 22, 23, 13, 15, 0, 24],
  [13, 15, 26, 27, 28, 29, 30],
  [31, 33, 34, 35, 37, 16, 38, 39, 41, 42, 22, 43, 46, 47, 48],
  [4, 32, 11, 15, 17, 49, 0, 50, 13],
  [4, 17, 51, 52, 5, 53, 43, 22, 54, 13, 45, 55],
  [40, 56, 5, 57, 43, 58],
  [60,
   61,
   6,
   62,
   9,
   10,
   64,
   15,
   65,
   37,
   17,
   66,
   0,
   67,
   69,
   40,
   71,
   56,
   72,
   73,
   53,
   74,
   75,
   77,
   78,
   2,
   79,
   14,
   80,
   82,
   83,
   84,
   45,
   85,
   20,
   28,
   88,
   48,
   90,
   7,
   92,
   95,
   96,
   97,
   5,
   98,
   25,
   99,
   100,
   22,
   43,
   58,
   102,
   13,
   104,
   106,
   107,
   1,
   24,
   108,
   30,
   109,
   110],
  [111, 69, 78, 112, 47, 13],
  [3,
   61,
   9,
   10,
   113,
   15,
   114,
   17,
   71,
   74,
   77,
   2,
   82,
   19,
   20,
   7,
   92,
   116,
   5,
   117,
   22,
   103,
   12,
  

In [37]:
dirname = os.path.dirname(os.path.abspath('ml-1m-3.pickle'))
os.makedirs(dirname, exist_ok=True)
with open('ml-1m-3.pickle', 'wb') as f:
    pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
user_list = [list() for u in train_df['user_id'].unique()]


In [25]:
len(user_list)

4250

In [27]:
for row in train_df.itertuples():
    print(row.user_id)

0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
19
19
19
19
19
19
19
19
19

460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
460
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
461
463
463
463
463
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
464
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
465
466
466
466
466
466
466
466
466
466
466
466
466
466
466
466
466
466
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
467
468
468
468
468
468
468
468
468
469
469
469
469
469
469
469
469
469
469
469


796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
796
797
797
797
797
797
797
797
797
797
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
798
799
799
799
799
799
799
799
799
799
799
799
799
799
799
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
802
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
803
804
804
804
804
804
804
804
804
804
804
804


1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1149
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1150
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1152
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153
1153


1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1471
1472
1472
1472
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1473
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1475
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476
1476


...

3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3710
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3711
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3712
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3713
3714
3714
3714
3714
3714


4050
4050
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4052
4058
4058
4058
4058
4058
4058
4058
4058
4058
4058
4058
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4060
4061
4061
4061
4061
4061
4061
4062
4062
4062
4062
4062
4062
4062
4062
4062
4062
4062
4062
4062
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4063
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065
4065


4450
4450
4450
4450
4450
4450
4450
4450
4450
4450
4450
4450
4450
4450
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4451
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4453
4455
4455
4455
4455
4455
4455
4455
4456
4456
4456
4456
4456
4456
4456
4456
4456
4456
4456
4456
4456
4456
4458
4458
4458
4459
4459
4459
4459
4459
4459
4460
4460
4460
4461
4461
4461
4461
4461
4462
4462
4462
4462
4462
4462
4462
4462
4462
4462
4462
4462
4462
4462
4463
4463
4463
4463
4463
4463
4463
4463
4463
4463
4463
4463
4463
4463
4463
4464
4464
4464


4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4869
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4870
4871
4871
4871
4871
4871
4871
4871
4871
4871
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4873
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4874
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4875
4876


5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5242
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5243
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5244
5245
5245
5245
5245
5245
5245
5245
5245
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5246
5248
5248
5248
5248
5248
5248
5248
5248
5248
5248
5248
5248
5248
5248
5249
5249
5249
5249
5249
5249
5249
5249
5249
5249
5249
5249


In [42]:
for row in df.itertuples():
    user_list[row.user_id].append(row.item_id)

1481

In [5]:
len(train_df['user_id'].unique())

6036

In [58]:
[1 for i in range(4)]

[1, 1, 1, 1]